In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import json
import os
import requests
from bs4 import BeautifulSoup

In [1]:
url = 'https://www.ncaa.com/rankings/soccer-men/d1/ncaa-mens-soccer-rpi'

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_conferences(url):
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            print("Error fetching the page.")
            return [], []

        # Create a BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')
        
        stats_table = soup.find('table')
        if not stats_table:
            print("Statistics table not found.")
            return [], []

        # Extract headers
        headers = [header.text.strip() for header in stats_table.find_all('th')]

        # Extract rows
        rows = []
        for row in stats_table.find_all('tr')[1:]:  # skip the header row
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            rows.append(cols)
        
        return headers, rows

    except requests.HTTPError as e:
        print(f'HTTP Error occurred: {e.response.status_code}')
        return [], []
    except requests.RequestException as e:
        print(f'Request exception: {e}')
        return [], []
    except Exception as e:
        print(f'An error occurred: {e}')
        return [], []

# Initialize an empty list to collect all the data
all_rows = []
all_headers = None

headers, rows = scrape_conferences(url)
    
if headers and rows:
    if all_headers is None:
        all_headers = headers  # Save headers once
    all_rows.extend(rows)  # Collect rows from all pages

if all_headers and all_rows:
    # Create a DataFrame from the concatenated data
    conferences = pd.DataFrame(all_rows, columns=all_headers)
else:
    print("No data was scraped.")
    
conferences = conferences[['School', 'Conference']]

In [12]:
conferences.to_csv('teams_conferences.csv')